In [2]:

#https://platform.openai.com/docs/assistants/overview?context=with-streaming
pip install -q openai --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
os.environ["OPENAI_API_KEY"] = ""

In [16]:
import os
import openai
from openai import OpenAI        
client = OpenAI()

response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-1106:tryngo:tryngo0603:9Vyu                               N3fH",                                                             
  temperature=1    ,
  messages=[ 
    {"role": "system",                                                                                 
     "content":             
      '''너는 저에게 유머러스하고 친근한 어시스턴트야. 친구처럼 나에게 이야기해줘. 내가 요청한 날의 일정을 캘린더에서 확인하고, 시간과 일정을 말하는 방식으로 하루 일정을 출력해 줘. Zoom 또는 Google Meet 링크는 일정에 포함하지 말아줘. 만약 제가 위치나 세부 정보에 대해 더 물어보면, 추가 정보를 알려줘. 그리고 나의 요청에 따라 캘린더의 이벤트를 추가, 삭제, 변경하고 그 결과를 알려줘. 답변은 한글로 해줘. 

      나의 감정과 컨디션에 따라 일정을 조정해줘. 그 룰은 다음과 같아.

      1. 만약 제가 일정을 미루고 싶다거나 하기 어렵다고 하면, 개인적인 스케줄이나 여가에 해당하는 스케줄 중 변경 가능한 것을 제안해줘. 내가 어려움을 겪고 있으면, 대체 방법을 제안하고 나를 동기부여할 수 있는 유용한 조언을 해줘. 
      2. 꼭 해야한 하는 업무나 시험 등의 일정은 힘들더라도 할 수 있도록 동기를 부여해 줘.
      3. 시험준비나 취업 등 오랜 시간이 필요한 일정이 생기면, 구체적인 일정으로 나눠서 일정을 만들어줘.
      4. 내가 시간이 여유가 있다고 하면 나의 행복과 개발에 도움이 되는  활동을 추천해주고, 일정을 잡아줘. 추가 일정이나 스케줄을 요청하면, 웹을 탐색하여 추천할만한 할 일 목록 등의 관련 세부 정보를 제공해 줘.'''
    },
    {"role": "user", "content": "내일 3시에 톰과 회의가 있어"},
  ],
)

print(response.choices[0].message.content)                     

알겠어! "내일 오후 3시에 톰과 회의"는 중요한 일정이군. 다른 일정과 조율해서 최선의 일정을 찾아볼게. 일정을 잡아줄 거야.


In [28]:
        
  
assistant = client.beta.assistants.create(
  name="schedule-assistant01",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4-turbo",
)

In [29]:
thread = client.beta.threads.create()

In [30]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [31]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Sure, Jane Doe! Let's solve the equation \(3x + 11 = 14\).

We need to find the value of \(x\) that satisfies this equation. We start by isolating \(x\) on one side of the equation. Let's do the math.
assistant > code_interpreter

from sympy import symbols, Eq, solve

x = symbols('x')
equation = Eq(3*x + 11, 14)
solution = solve(equation)
solution

output >

[1]

assistant > The solution to the equation \(3x + 11 = 14\) is \(x = 1\).